In [2]:
from utils import *
import pandas as pd
import os

In [ ]:
### ipynb에서는 os.getenv 사용 불가(저는 직접 넣어 사용하고 지웠습니다)
model = Upstage(api_key=os.getenv("UPSTAGE_API_KEY"))

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [4]:
### request를 무조건 list(dict{role, content})로 전달해야함
temp_request = [{"role": "user", "content": "안녕"}]
print(model.summary(temp_request))

안녕하세요! 😊  
어떻게 도와줄 수 있을까요? 궁금한 점이나 도움이 필요한 것이 있다면 편하게 알려주세요.  

예를 들어...  
- **질문**이 있거나,  
- **추천**이 필요하거나,  
- **간단한 대화**를 나누고 싶다면 언제든 말씀해주세요!  

기다리고 있을게요 ✨


In [17]:
df = pd.read_csv("./korean_jd_105.csv")

In [6]:
def format_jd_to_prompt_short(jd_data):
    items = [f"{k}: {v}\n" for k, v in dict(jd_data).items() if str(v).strip() not in ('', 'nan', 'NaN', 'None')]
    return '\n'.join(items).strip()

In [18]:
jd_prompts = []
for i in range(len(df)):
    jd_prompts.append(format_jd_to_prompt_short(df.iloc[i][['url', 'job_name', 'title', 'company_name', 'location', 'experience_requirement',
       'description', 'main_work', 'qualification',
       'preferences', 'welfare']]))
    print(f"{i} processed")   

0 processed
1 processed
2 processed
3 processed


In [ ]:
system_prompt = """당신은 JD를 분석하여 정형화된 요약본을 생성하는 인공지능 비서입니다. 제공된 모든 JD 정보를 분석하여, 다음 3가지 섹션을 포함하는 마크다운 형식의 한국어 요약본을 생성해야 합니다: 1. 주요 직무 및 목표, 2. 필수 역량 및 자격, 3. 우대 사항 및 복지. 각 섹션의 내용은 불릿 리스트(-)로 구성하고, 원문에서 제시된 내용만 사용하며 추측성 내용을 배제하세요.
"""

In [43]:
# print(jd_prompts[1])

In [41]:
### request를 무조건 list(dict{role, content})로 전달해야함
request = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": jd_prompts[1]}
]
print(model.summary(request))

### 📌 **AI 소프트웨어 풀스택 개발자 (3~12년) 요약**  

#### 1. **주요 직무 및 목표**  
- **LLM 기반 시스템 개발**: RAG 시스템 및 Agent 개발  
- **AI Orchestration 구현**: Multi LLM, STT, TTS 등 도메인 특화 Agent 개발  
- **시스템 아키텍처 설계**: SaaS 및 온프레미스 환경에서의 설계, 배포, 운영  
- **프롬프트 엔지니어링**: CoT, ReAct 등 기법 활용 및 LLM 기능 확장 모듈 개발  
- **데이터 파이프라인 구축**: MQ, Stream 활용 및 VectorDB/RDBMS/NOSQL 기반 시각화  
- **운영 플랫폼 개발**: 최적화 플랫폼 및 운영 툴 개발  

#### 2. **필수 역량 및 자격**  
- **AI/ML 기술 이해**: 딥러닝 및 LLM 관련 지식 보유  
- **프로토콜 이해**: MCP, A2A 등 통신 프로토콜 경험  
- **웹 애플리케이션 개발**: REST API 설계·개발·운영 경험  
- **풀스택 기술 스택**:  
  - Backend: Spring Boot, FastAPI 등  
  - Frontend: React, Typescript 등  
- **데이터베이스 관리**: RDBMS/NOSQL/VectorDB 설계 및 SQL 최적화 경험  
- **클라우드 활용**: AWS/GCP/Azure 서비스 배포 경험  
- **DevOps 경험**: Docker/Kubernetes 기반 서비스 운영 및 자동화  

#### 3. **우대 사항 및 복지**  
- **우대 사항**  
  - 기술적 커뮤니케이션 능력 및 팀웍  
  - 다양한 분야에 대한 호기심과 빠른 학습 능력  
  - 컴퓨터공학 전공 또는 IT 관련 자격증 보유  
  - 대규모 검색/벡터 검색 시스템 구축 경험  
  - OpenAI/Hugging Face 등 외부 LLM 연동 경험  
  - MLOps 환경 구축·운영 경험 (모델 버

# 모듈화

In [7]:
import yaml

In [8]:
df = pd.read_csv("./korean_jd_105.csv")

In [ ]:
model = Upstage(api_key=os.getenv("UPSTAGE_API_KEY"))

In [10]:
with open("./prompts.yaml", 'r', encoding='utf-8') as file:
        prompts = yaml.safe_load(file)

In [11]:
# print(prompts["prompts_ver1"]["user"].format(jd=jd_prompts[1]))

In [12]:
from tqdm import tqdm

In [19]:
### request를 무조건 list(dict{role, content})로 전달해야함
summaries = []
for i in tqdm(range(len(df))):
    request = [
        {"role": "system", "content": prompts["prompts_ver1"]["system"]},
        {"role": "user", "content": prompts["prompts_ver1"]["user"].format(jd=jd_prompts[i])}
    ]  
    summaries.append(model.summary(request))

100%|██████████| 4/4 [00:15<00:00,  3.85s/it]


In [90]:
summaries = summaries*51
summaries = summaries[:len(df)]

In [91]:
df["summary"] = summaries

In [20]:
summaries

['### 주요 직무 및 목표  \n- LiveKlass 서비스 백엔드 시스템 설계 및 개발  \n- 프론트엔드 팀과 협업하여 확장성 및 성능 우수한 시스템 구축  \n- 스쿼드 단위(PM, UXUI, FE 팀 등)로 기능 개발 및 협업  \n- AWS 클라우드 환경에서 서비스 개발, 모니터링, 운영 및 DevOps/SRE 문화 구축  \n- Jira, Slack, Notion, Github 등 협업 툴 활용 및 비즈니스 팀과 지속적 협업  \n- 신기술 학습 및 문제 해결을 위한 협력  \n\n### 필수 역량 및 자격  \n- 5년 이상 백엔드 개발 경력 또는 이에 상응하는 경험  \n- 대규모 트래픽 환경에서의 서비스 운영 및 성능 최적화 경험  \n- 스쿼드 단위 과제 주도 및 백엔드 아키텍처 설계·구현·운영 리딩 능력  \n- **Java & Spring Framework**: Java 8+, Spring Boot, Spring MVC, Spring Data JPA, Spring Security 활용 능력  \n- **데이터베이스 (MariaDB)**: MariaDB 설계·최적화, ORM(JPA/Hibernate) 활용, Native Query 튜닝, 데이터 마이그레이션/백업 경험  \n- **클라우드 & DevOps (AWS, Docker)**: AWS 서비스(EC2, S3, RDS 등), Docker 컨테이너/Docker Compose, ECS/EKS/K8S 운영, CI/CD 파이프라인(GitHub Actions/Jenkins) 구축 경험  \n- RESTful API 설계·개발, 대규모 트래픽 고려 시스템 설계, Git 협업, 단위/통합 테스트(JUnit/Mockito) 작성, 타 직군 협업 경험  \n\n### 우대 사항 및 복지  \n**우대 사항**  \n- 커머스 플랫폼 또는 신규 서비스 백엔드 아키텍처 설계·구현 경험  \n- 코드 리뷰/기술 세션을 통한 팀 내 기술 기준 수립 및 공유 경험  \n- DevOps, 테스트 자동화, 모니터

In [16]:
data = preprocess(data)

전처리를 시작합니다
문장처리&청킹 완료
원본 데이터 개수: 105
청크 개수: 525


In [18]:
data[0]

Document(metadata={'description': "Mission : 우리는 지식으로 자신과 세상의 변화를 꿈꾸는 사람들을 연결합니다. Vision : 사는 곳이 배움의 기회를 제한하지 않는 세상을 만듭니다. [지식으로 세상을 연결하는 라이브클래스] 우리는 위 미션을 바탕으로 비전을 이루기 위해 ‘라이브클래스’와 '라이브클래스Pro'를 만들고 있습니다. ‘라이브클래스’는 나만의 온라인 강의 사이트를 손쉽게 만들 수 있는 지식 비즈니스 플랫폼으로 사람들이 자신의 지식을 가장 쉽고 빠르게 판매할 수 있도록 돕습니다. '라이브클래스Pro'는 대규모의 챌린지 또는 코호트 교육을 하는 기업에 최적화된 플랫폼으로 전용 앱과 웹을 제공합니다. [국내 1위 지식 비즈니스 플랫폼, 라이브클래스] • 2만 여개의 고객사, 월 50만명이 사용하는 플랫폼 • 창업 2년 만에 월 BEP 달성, 연 평균 500%의 거래액 성장 기록 • 정부 지자체, 대학, 민간기관 등 100여곳의 기관에서 활용하는 라이브클래스 • 70억 이상의 누적 투자 유치(뮤렉스파트너스, 티비티파트너스, HB인베스트먼트, 임팩트스퀘어 등) • 대한민국 대표 강사 김미경(MKYU)의 '미모리안' 등 각 분야별 1위 기업 및 지식 크리에이터들을 통한 성장 [우리는 이렇게 일하고 성장해요.] 1. 막연한 불안과 걱정 대신, 내가 바꿀 수 있는 것(영향력의 원)에 집중해요. 2. 고객이 다섯을 원할 때, 열을 만족 시키기 위해 노력해요. 3. 빠르게 실행하고, 데이터에 기반해 집요하게 개선해요. 4. 최고의 서비스를 만들기 위해 한계를 두지 않고 지속적으로 성장해요. 5. No라고 말하는 것을 두려워하지 않고, 결국 Yes를 이끌어 내요. 6. 상대방을 존중하되, 대안을 가지고 솔직하게 소통해요. 7. 미친 가속을 원하는 사람도, 워라밸을 원하는 사람도 상호 성장하는 문화를 만들어 가요. 8. 현재 내가 누리는 것에 감사하며, 대접 받고 싶은 대로 먼저 실천해요. 라이브클래스와 세상을 바꾸는 여정에 함께할 당